In [ ]:
import gradio as gr
import os

import pickle

from NN_prediction import neural_net_predictor          #Python files used for Task 1
from RF_prediction import random_forest_predictor
from XGBoost_prediction import xgboost_predictor

from BERT_NN_GUI import bert_nn                         #Python files used for Task 2
from BERT_RF_GUI import bert_rf
from BERT_XGBOOST_GUI import bert_xgboost


source_options = ["My Own Dataset", "Pre-Allocated Dataset"]
task_options = ["Emotion Identification", "Positive/Neutral/Negative", "Star Rating"]       #Here we define the 4 relevant parameters for task 1: data source, the task, classification method and the pre-trained data used
classification_options = ["Neural Net", "Random Forest", "XGBoost"]
Pickle_options = ["twit_df_7852","reddit_df_7852","imdb_df_7852","yelp_df_7852", "My own data"]


# Code for Task 1 (Predictions using different classifiers and trained models for 3 different tasks)
def task1(source, task, classification, trained_model, text):
    
    if source == "Pre-Allocated Dataset":         
        if classification == "Neural Net":                                                  #For Neural Net:
            with open(f'Pickle/Neural Net/{trained_model}.pickle', 'rb') as handle:         #Opens pickle file containing the specified pre-trained model
                pre_trained_model = pickle.load(handle)
            return neural_net_predictor(task, pre_trained_model, text)                             #Runs neural net predictor function
        
        elif classification == "Random Forest":                                             #For Random Forest:
            with open(f'Pickle/Random Forest/{trained_model}.pickle', 'rb') as handle:
                pre_trained_model = pickle.load(handle)
            return random_forest_predictor(task, pre_trained_model, text)                          #Runs random forest predictor function
        
        elif classification == "XGBoost":                                                   #For XGBoost:
            with open(f'Pickle/XGBoost/{trained_model}.pickle', 'rb') as handle:
                pre_trained_model = pickle.load(handle)
            return xgboost_predictor(task, pre_trained_model, text)                                #Runs xgboost predictor function
    

    if source == "My Own Dataset":
        
        if classification == "Neural Net":  
            trained_model = os.listdir('Your_Pickle/Neural Net')[0]                                                #For Neural Net:
            with open(f'Your_Pickle/Neural Net/{trained_model}', 'rb') as handle:         #Opens pickle file containing the specified pre-trained model
                pre_trained_model = pickle.load(handle)
            return neural_net_predictor(task, pre_trained_model, text)                             #Runs neural net predictor function
        
        elif classification == "Random Forest":                                                     #For Random Forest:
            trained_model = os.listdir('Your_Pickle/Random Forest')[0]                                               
            with open(f'Your_Pickle/Random Forest/{trained_model}', 'rb') as handle:
                pre_trained_model = pickle.load(handle)
            return random_forest_predictor(task, pre_trained_model, text)                          #Runs random forest predictor function
        
        elif classification == "XGBoost":                                                   #For XGBoost:
            trained_model = os.listdir('Your_Pickle/XGBoost')[0]
            with open(f'Your_Pickle/XGBoost/{trained_model}', 'rb') as handle:
                pre_trained_model = pickle.load(handle)
            return xgboost_predictor(task, pre_trained_model, text)                                #Runs xgboost predictor function



# Code for task 2
def task2(classification, uploaded_file):

    if classification == "Neural Net":
        return bert_nn(rf"{uploaded_file}")     #trains the given datafile with the corresponding classification method
    
    elif classification == "Random Forest":
        return bert_rf(rf"{uploaded_file}")
    
    elif classification == "XGBoost":
        return bert_xgboost(rf"{uploaded_file}")


# Interface one, Prediction
iface1 = gr.Interface(
    fn=task1,

    inputs=[gr.Dropdown(choices = source_options), gr.Dropdown(choices = task_options),gr.Dropdown(choices = classification_options), gr.Dropdown(choices = Pickle_options), "text"],
    outputs="text",

    title=  "<div style='text-align: center;'>"
                "<span style='font-family: \"Times New Roman\", sans-serif; font-style: italic; font-weight: bold;'>Sentiment Predictor</span>"
                "<p style='font-size: 16px;'>Predict sentiments using different trained models and classifiers </p>"
                "<p style='font-size: 14px;'>Warning: as of current release, you are able to use any trained model for any task. However, for accurate results, please use twitter for emotion prediction, IMDB or reddit for positive/negative/neutral, and yelp for star ratings. Please note however that if IMDB is chosen for positive/negative/neutral task, it will only return postive or negative</p>"
            "</div>"
)


# Interface two, Train Your own Model
iface2 = gr.Interface(
    fn=task2,

    inputs= [gr.Dropdown(choices = classification_options), gr.UploadButton(file_types=[".csv", ".tsv", ".parquet"])],
    outputs="image",

    title=  "<div style='text-align: center;'>"
                "<span style='font-family: \"Times New Roman\", sans-serif; font-style: italic; font-weight: bold;'>Train Your own Model</span>"
                "<p style='font-size: 16px;'>Train a model to predict sentiments from a data file that you upload yourself!</p>"
                "<p style='font-size: 14px;'>Warning: To train a model on your own dataset, please have two columns named 'text' and 'label' in order for the program to function as intended. Possible file extensions are .csv .tsv .parquet </p>"
                "<p style='font-size: 14px;'>Warning: DO NOT train more than 1 dataset for a given classification method. As of current release, if you want to run predictions on your own dataset, the program will only read the first datafile uploaded for a given classification method. If you wish to use a new datafile, you will have to go into the 'Your_Pickle' folder and delete the previous datafile uploaded </p>"
                "<p style='font-size: 14px;'>Warning: You may need to upload the data file twice. For some unkown reason, sometimes the first time you upload the file it gets registered as 'None'</p>"
            "</div>"
)

demo = gr.TabbedInterface([iface1, iface2], ["Sentiment Prediction", "Train your Own Model"])   #Naming the two pages and definining the final demo

# Run the interface
demo.launch(share=True)